In [1]:
from cad import CAD
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name = "/root/share/chinese-alpaca-2-13b"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, )
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, padding_side="left", )
cad = CAD(model=model, tokenizer=tokenizer)

/opt/conda/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-10 01:17:57,213] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.58s/it]
/opt/conda/envs/LLM/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/LLM/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  wa

In [2]:
import pandas as pd

PROMPT_TEMPLATE = """[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

【已知】：【"项目名称"：{title}
{label}：{content}】
你现在是一个政企领域的专家，基于【已知】给的多个内容，请回答问题：“{question}”，一定要保证答案正确！[/INST]"""

df = pd.read_json("/root/share/scripts/output/intent-chatglm3-6b-optimize.json")
row = df.loc[5]
question = row["一般提问"]
prompt = PROMPT_TEMPLATE.format(
    title=row["title"],
    label=row["二级policy预测"],
    content=row["content"],
    question=row["一般提问"],
)

print(prompt)

[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

【已知】：【"项目名称"：深圳市市场监督管理局宝安监管局2022年度知识产权优秀服务机构奖励项目
基本信息：政策项目类型: 评审类
政策项目审批类型: 人工
是否启动申报: 已启动
发布范围: 公开发布
办理时限: 90
受理部门: 市市场监督管理局宝安监管局
支持方式: 资金
项目实施地点: 宝安区
窗口收件是否需到大厅提交材料: 否
是否项目公示: 是
实施年度: 2022
申报条件: （一）具备独立法人资格，在宝安区开展实际经营活动的企业或机构；
（二）拥有专利代理师或律师6名以上；
（三）上年度代理专利诉讼案件达5件以上或上年度代理的发明专利授权数量达100件以上，且至少为2家宝安区的国家专精特新“小巨人”企业、省“专精特新”企业或宝安区六类百强企业提供知识产权业务服务；
（四）知识产权专业化、市场化、国际化服务工作取得显著成效，具备一定的知识产权服务出口能力；
（五）本年度未获得市知识产权服务机构类资助。
奖励标准: 每年评选不超过5家知识产权优秀服务机构，每家奖励30万元。
申请材料: （一）项目申请表；
（二）法定代表人授权委托证明书、银行开户证明（可导入）；
（三）国家知识产权局授予发放的批准设立机构通知书及专利代理机构注册证复印件；
（四）专业工作人员清单、近3个月社保缴纳费用明细表（明细表中要能清晰展示社保缴纳单位的全称）及相应的执业资格证明文件；
（五）上年度为宝安区企事业单位、其他机构或个人提供知识产权服务清单；
（六）知识产权业务相关材料，包括但不限于专利诉讼代理合同、诉讼判决文书、知识产权服务合同、代理授权专利清单、发明专利代理授权率相关佐证资料等（外文资料需提供中文翻译件）；
（七）知识产权专业化、市场化、国际化服务工作开展情况总结及证明材料；
（八）上年度经营状况良好，产生了较好的经济效益的相关佐证材料；
（九）其他必要的申请材料。
项目审批补充流程: 1.材料评审：深圳市市场监督管理局宝安监管局组织专家对材料进行评审，根据专家评分排序，确定进入现场评审单位名单。
2..现场评审：深圳市市场监督管理局宝安监管局对入围单位进行现场核查，根据专家综合评分（材料评审与现场评审得分之和）排序确定拟奖名单。
受理

In [3]:
outputs_no_cad = cad.generate(
    texts=[prompt],
    max_new_tokens=200,
)
print(outputs_no_cad)

['深圳市市场监督管理局宝安监管局2022年度知识产权优秀服务机构奖励项目的发布范围是公开发布的。']


In [5]:
outputs = cad.generate(
    texts=[question],
    texts_with_context=[prompt],
    max_new_tokens=200,
    alpha=0.5,
)

print(outputs)

['“公开发布”']


In [6]:
from tqdm import tqdm

tqdm.pandas()

df["reply_alpaca2_cad"] = df.progress_apply(
    lambda row: cad.generate(
        texts=[row["一般提问"]],
        texts_with_context=[PROMPT_TEMPLATE.format(
            title=row["title"],
            label=row["二级policy预测"],
            content=row["content"],
            question=row["一般提问"],
        )],
        max_new_tokens=2048,
        alpha=0.5,
    )[0],
    axis=1,
)

100%|██████████| 100/100 [17:47<00:00, 10.68s/it]


In [ ]:
df.to_excel("output/alpaca2_cad.xlsx")